In [1]:
!python.exe -m pip install --upgrade pip
!pip install --upgrade python-telegram-bot

/bin/bash: line 1: python.exe: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.6 MB/s eta 0:00:00


In [2]:
!pip install nest_asyncio

In [3]:
pip install pytz

In [ ]:
#library
import datetime
from telegram import Update
from telegram.ext import Application, CommandHandler, ContextTypes, MessageHandler, filters
import logging
from typing import Final
import random
import time

# Setup
TOKEN: Final = '7391587311:AAESm_hGgbtFVzDixgE8a0ArAtOxgRrB4Lc'
BOT_USERNAME: Final = '@SeminarSeminarBot'
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)


# list of seminars category
seminars = {
    "musical": [],
    "educational": [],
    "health and Wellness": [],
    "business": [],
    "art": []
}



#start Function
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('Hello! I am Seminar Bot, How can i help you...')

#help
async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('Welcome to the Seminar Bot! Use /addseminar to add a new seminar, and /display to see all available seminars!')





#categories to choose from
def list_categories():
    return ', '.join(seminars.keys())


#add seminar
async def add_seminar(update: Update, context: ContextTypes.DEFAULT_TYPE):
    context.user_data['step'] = 'choose_category'
    await update.message.reply_text(f'Please choose a category to add a seminar: {list_categories()}')

async def choose_category(update: Update, context: ContextTypes.DEFAULT_TYPE):
    category = update.message.text.strip()
    if category in seminars:
        context.user_data['add_category'] = category
        context.user_data['step'] = 'add_details'
        await update.message.reply_text("Please provide details like this: 'Title, Location, YYYY-MM-DD, HH:MM, URL, Communication method'")
    else:
        await update.message.reply_text(f'Invalid category. Please choose from: {list_categories()}')



async def save_seminar_details(update: Update, context: ContextTypes.DEFAULT_TYPE):
    category = context.user_data['add_category']
    try:
        details = update.message.text.split(', ')
        if len(details) != 6:
            await update.message.reply_text("Please provide all details like this: 'Title, Location, YYYY-MM-DD, HH:MM, URL, Communication method'")
            return

        seminar = {
            'name': details[0],
            'location': details[1],
            'date': details[2],
            'time': details[3],
            'url': details[4],
            'Communication method': details[5]
        }

        seminars[category].append(seminar)
        await update.message.reply_text(f'Seminar "{details[0]}" added successfully in category {category}.')
        context.user_data.pop('add_category')
        context.user_data.pop('step')
    except Exception as e:
        await update.message.reply_text(f'Error adding seminar: {e}')




# Display seminars
async def display_seminars(update: Update, context: ContextTypes.DEFAULT_TYPE):
    remove_seminar()
    if not any(seminars.values()):
        await update.message.reply_text('There are no available seminars.')
        return

    display = ""
    for category, seminar_list in seminars.items():
        if seminar_list:
            display += f"\n*{category} Seminars:*\n"
            for i, seminar in enumerate(seminar_list, start=1):
                display += (
                    f"  {i}. *Title:* {seminar['name']}\n"
                    f"     *Location:* {seminar['location']}\n"
                    f"     *Date:* {seminar['date']}\n"
                    f"     *Time:* {seminar['time']}\n"
                    f"     *URL:* {seminar['url']}\n"
                    f"     *Communication method:* {seminar['Communication method']}\n\n"

                )

    await update.message.reply_text(display, parse_mode='Markdown')





#remove outdated seminars
def remove_seminar():
    current_date = datetime.datetime.now()
    for category in seminars:
        seminars[category] = list(filter(lambda seminar: datetime.datetime.strptime(f"{seminar['date']}, {seminar['time']}", '%Y-%m-%d, %H:%M') >= current_date, seminars[category]))




# Basic
# Function to process and respond to text
def handle_response(text: str) -> str:
    processed: str = text.lower()
    if 'seminar' in processed:
        return 'Use /display command to see available seminars.'
    return 'I do not understand you.'

# Message handler
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if context.user_data.get('step') == 'choose_category':
        await choose_category(update, context)
    elif context.user_data.get('step') == 'add_details':
        await save_seminar_details(update, context)
    else:
        message_type: str = update.message.chat.type
        text: str = update.message.text

        print(f'User ({update.message.chat.id}) in {message_type} chat: "{text}"')

        if message_type == 'group':
            if BOT_USERNAME in text:
                new_text: str = text.replace(BOT_USERNAME, '').strip()
                response: str = handle_response(new_text)
            else:
                return
        else:
            response: str = handle_response(text)

        print('Bot:', response)
        await update.message.reply_text(response)

def main():
    print('Starting bot')
    app = Application.builder().token(TOKEN).build()

    # Commands
    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("help", help_command))
    app.add_handler(CommandHandler("addseminar", add_seminar))
    app.add_handler(CommandHandler("display", display_seminars))

    # Message handler
    app.add_handler(MessageHandler(filters.TEXT & (~filters.COMMAND), handle_message))

    # Error handler
    # app.add_error_handler(error)  # Uncomment when function is defined

    # Start polling
    print('Polling...')
    app.run_polling(poll_interval=3)

if __name__ == '__main__':
    import nest_asyncio
    nest_asyncio.apply()
    main()

Starting bot
Polling...
